In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
import json
from PIL import Image
from base64 import b64encode,b64decode
from io import BytesIO

df = pd.read_csv('account.csv')

In [4]:
opts = webdriver.ChromeOptions()
#opts.add_argument("--incognito")

ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
opts.add_argument("user-agent={}".format(ua))
##opts.add_argument("--headless")       # define headless
driver = webdriver.Chrome(chrome_options=opts)
driver.set_window_size(1000, 800)
fail_list = []

for account, password in df.values :
    print("Voting with ",account," !")
    complete = False
    vote_flag = False
    count = 0
    login_flag = False
    while complete == False:
        try:
            driver.get("https://youthdream.phdf.org.tw/member/login")
            sleep(5)
            ## Login
            while login_flag == False :
                try:
                    ## get captcha picture
                    driver.execute_script("""
                        (function () {
                            var y = 0;
                            var step = 100;
                            window.scroll(0, 9999);
                        })();
                    """)
                    
                    captcha = driver.find_element_by_id('captcha')
                    offset = 1302
                    driver.save_screenshot('./screen_shot/captcha.png')
                    left = captcha.location['x']+1
                    top = captcha.location['y']+1-offset
                    right = captcha.location['x'] + captcha.size['width']
                    bottom = captcha.location['y'] + captcha.size['height']-offset
                    im = Image.open('./screen_shot/captcha.png')
                    im = im.crop((left, top, right, bottom))
                    rgb_im = im.convert('RGB')
                    re = rgb_im.resize((160,60))
                    re.save('./screen_shot/captcha/captcha.jpg')
                    sleep(2)
                    
                    ## break captcha
                    
                    ENCODING = 'utf-8'
                    output_buffer = BytesIO()
                    re.save(output_buffer, format='JPEG')
                    byte_data = output_buffer.getvalue()
                    base64_bytes = b64encode(byte_data)
                    base64_string = base64_bytes.decode(ENCODING)
                    data = {"Captcha":base64_string}  # img是ndarray，無法直接用base64編碼，否則會報錯
                    data = json.dumps(data)
                    r = requests.post(" https://captchabreak-261615.appspot.com/Captcha", data=data)
                    c = json.loads(r.text)["Captcha"]
                    
                    ##login
                    driver.find_element_by_name("email").clear()
                    driver.find_element_by_name("email").send_keys(account)
                    driver.find_element_by_name("password").clear()
                    driver.find_element_by_name("password").send_keys(password)
                    driver.find_element_by_name("captcha").clear()
                    driver.find_element_by_name("captcha").send_keys(c)
                    driver.find_element_by_xpath("//button[@class='button-5']").click()
                    sleep(2)
                except:
                    try :
                        element = driver.find_element_by_name("email")
                    except:
                        print("sign in success!")
                    login_flag = True
                try :
                    element = driver.find_element_by_name("email")
                except:
                    print("sign in success!")
                    login_flag = True
'''   投票已經結束了
            while vote_flag == False:
                page = driver.page_source
                priceValue = driver.find_element_by_xpath("//meta[@name='csrf-token']")

                cookies = driver.get_cookies()
                cookie = [item["name"] + "=" + item["value"] for item in cookies ]
                cookiestr = ';'.join(item for item in cookie)
                headers = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0",
                    "X-CSRF-TOKEN":str(priceValue.get_attribute("content")),
                    "Accept": "*/*",
                    "Connection": "keep-alive",
                    "Content-Length": "0",
                    "Cookie": cookiestr,
                    "Host": "youthdream.phdf.org.tw",
                    "Origin":"https://youthdream.phdf.org.tw",
                    "Referer": "https://youthdream.phdf.org.tw/project/show/83",
                    "Sec-Fetch-Mode": "cors",
                    "X-Requested-With": "XMLHttpRequest"
                }
                print("voting to 蘭友")
                r = requests.post("https://youthdream.phdf.org.tw/project/83",headers = headers)
                print("voting to 南友")
                r2 = requests.post("https://youthdream.phdf.org.tw/project/82",headers = headers)
                if r.status_code == 200 and r.status_code == 200:
                    vote_flag =True
                    print(account," message = ",json.loads(r.text))
                else :
                    print("Vote again")
            sleep(2)
            
''' 
            
            ## logout google 
            driver.get("https://youthdream.phdf.org.tw/member/logout")
            sleep(2)
            complete = True
            sleep(5)
        except:
            count += 1
            if count == 5 :
                complete = True
                fail_list.append(account)
            else:
                print(account,"Try again")
            sleep(10) 
            driver.get("https://youthdream.phdf.org.tw/member/logout")
            sleep(2)
        

print("Complete")
sleep(5)
driver.close()

c:\users\user\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


Voting with  lingqil586@gmail.com  !
Complete
